# Computing for Business Research - Homework Assignment #2 

Name: Sin Ting Cindy Yu

UNI: sy3163

## Question 1 ##

### Part 1

In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin

In [3]:
seed_url = "https://press.un.org/en"

urls = [seed_url]
seen = [seed_url]
opened = []

pr_list = []

max_pr = 10

while len(urls) > 0 and len(pr_list) < max_pr:
    try:
        curr_url = urls.pop(0)
        response = requests.get(curr_url, headers={'User-Agent': 'Mozilla/5.0'})
        webpage = response.content
        opened.append(curr_url)
        
    except:
        continue
        
    soup = BeautifulSoup(webpage)
    check_pr = soup.find("a", {"href":"/en/press-release", "hreflang":"en"}, text="Press Release")
    check_page = soup.find("h1", text="Press Release")
    if check_pr is not None:
        press_release = soup.find(class_="col-md-9 mb-2 panel-panel radix-layouts-main-column").text
        if "crisis".lower() in press_release.lower():
            pr_list.append(curr_url)
    elif (curr_url==seed_url) or (check_page is not None):
        for tag in soup.find_all("a", href = True):
            childUrl = tag["href"]
            childUrl = urljoin(curr_url, childUrl)
            if childUrl not in seen:
                urls.append(childUrl)
                seen.append(childUrl)

C:\Users\User\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [4]:
pr_list

['https://press.un.org/en/2023/sgsm21996.doc.htm',
 'https://press.un.org/en/2023/sgsm21994.doc.htm',
 'https://press.un.org/en/2023/sgsm21993.doc.htm',
 'https://press.un.org/en/2023/gapal1455.doc.htm',
 'https://press.un.org/en/2023/sgsm21991.doc.htm',
 'https://press.un.org/en/2023/sgsm21990.doc.htm',
 'https://press.un.org/en/2023/sgsm21987.doc.htm',
 'https://press.un.org/en/2023/sgsm21985.doc.htm',
 'https://press.un.org/en/2023/sgsm21982.doc.htm',
 'https://press.un.org/en/2023/sgsm21980.doc.htm']

### Part 2

In [4]:
!pip install selenium

In [5]:
from selenium import webdriver
import time

In [8]:
driver = webdriver.Chrome()
seed_url = "https://www.europarl.europa.eu/news/en/press-room"

urls = []
seen = set()
opened = []

pr_list = []

max_pr = 10

driver.get(seed_url)
main_page = driver.page_source
main_soup = BeautifulSoup(main_page)
for tag in main_soup.find_all(class_="ep_gridcolumn ep-m_product ep-layout_linkmode"):
    url = tag.find("a")["href"]
    if url not in seen:
        urls.append(url)
        seen.add(url)

while len(pr_list) < max_pr:
    
    if len(urls) > 0:
    
        try:
            curr_url = urls.pop(0)
            response = requests.get(curr_url, headers={'User-Agent': 'Mozilla/5.0'})
            webpage = response.content
            opened.append(curr_url)

        except:
            continue

        soup = BeautifulSoup(webpage)
        check_plenary = soup.find("span", {"class":"ep_name"}, text="Plenary session")
        if check_plenary is not None:
            heading = soup.find("h1")
            date = soup.find(class_="ep-p_text ep-layout_date")
            bullets = soup.find(class_="ep-a_facts")
            summary = soup.find(class_="ep-a_text ep-layout_chapo")
            content = soup.find_all(class_="ep-a_text")[1]
            pr = {}
            if heading is not None:
                pr['Heading'] = heading.text
            if date is not None:
                pr['Date'] = date.text
            if bullets is not None:
                pr['Bullets'] = bullets.text
            if summary is not None:
                pr['Summary'] = summary.text
            if content is not None:
                pr['Content'] = content.text
            if any("crisis".lower() in value.lower() for value in pr.values()):
                pr_list.append(curr_url)
    
    elif (len(urls) == 0) and (main_soup.find(id="continuesLoading_button") is not None):
        
        load_more = driver.find_element("id", "continuesLoading_button")
        driver.execute_script('arguments[0].click()', load_more)
        time.sleep(5)
        main_page = driver.page_source
        main_soup = BeautifulSoup(main_page)
        for tag in main_soup.find_all(class_="ep_gridcolumn ep-m_product ep-layout_linkmode"):
            url = tag.find("a")["href"]
            if url not in seen:
                urls.append(url)
                seen.add(url)

driver.quit()

In [9]:
pr_list

['https://www.europarl.europa.eu/news/en/press-room/20231013IPR07129/eu-budget-2024-addressing-next-year-s-challenges-through-mid-term-revision',
 'https://www.europarl.europa.eu/news/en/press-room/20230929IPR06132/nagorno-karabakh-meps-demand-review-of-eu-relations-with-azerbaijan',
 'https://www.europarl.europa.eu/news/en/press-room/20230929IPR06130/parliament-argues-for-a-top-up-to-multi-annual-budget-for-crisis-response',
 'https://www.europarl.europa.eu/news/en/press-room/20230911IPR04923/reduce-demand-and-protect-people-in-prostitution-say-meps',
 'https://www.europarl.europa.eu/news/en/press-room/20230911IPR04918/svietlana-tsikhanouskaya-to-meps-support-belarusians-european-aspirations',
 'https://www.europarl.europa.eu/news/en/press-room/20230911IPR04908/meps-vote-to-strengthen-eu-defence-industry-through-common-procurement',
 'https://www.europarl.europa.eu/news/en/press-room/20230707IPR02427/covid-19-parliament-adopts-roadmap-to-better-prepare-for-future-health-crises',
 'htt